<a href="https://colab.research.google.com/github/kartulle/Codigo-QR/blob/main/1_2_(Opcional)_Classifica%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🐱🐶 Classificação de Imagens (Cats vs Dogs) com PyTorch — **Notebook de Referência**
**Objetivo:** treinar um classificador binário (gato vs. cachorro) usando **`torchvision.datasets.OxfordIIITPet`**, que já está disponível para *download automático*.

**Como usar:**
- As únicas linhas que serão avaliadas pelos instrutores estão entre `### Inicie seu código aqui ###` e `### Termine seu código aqui ###`.
- Cada participante pode adicionar células em qualquer ponto do _notebook_ para fins de experimentação da forma mais conveniente possível.
- Há *sanity checks* ao longo do notebook para verificar se cada etapa está correta.
- O caminho de resolução está guiado: baixar dados → preparar *DataLoaders* → definir modelo → treinar → avaliar.



## 1. Ambiente e versões
Se estiver no Google Colab, certifique-se de que a GPU está habilitada em **Runtime → Change runtime type → GPU**.


In [ ]:

# Versões principais (apenas para log)
import torch, torchvision, sys
print("Python:", sys.version)
print("PyTorch:", torch.__version__)
print("Torchvision:", torchvision.__version__)

# Verificação de GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
assert device.type in ["cuda", "cpu"]



## 2. Imports, semente aleatória e utilitários


In [ ]:

import os, random, math, time
from dataclasses import dataclass
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, Subset
import torchvision
from torchvision import transforms
from torchvision.datasets import OxfordIIITPet
import numpy as np

# Reprodutibilidade
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Sanity-check helper
def assert_true(condition, msg_ok="OK", msg_fail="Falhou"):
    if condition:
        print("✅", msg_ok)
    else:
        raise AssertionError("❌ " + msg_fail)



## 3. Hiperparâmetros (edite aqui se desejar)


In [ ]:

# Hiperparâmetros principais
BATCH_SIZE = 32
LR = 1e-3
EPOCHS = 3
IMG_SIZE = 224
VAL_SPLIT = 0.2
NUM_WORKERS = 2 if torch.get_num_threads() > 1 else 0

# (Opcional) Subamostragem para testes rápidos: use None para dataset completo
TRAIN_MAX_SAMPLES = None   # e.g., 200 para rodar mais rápido
VAL_MAX_SAMPLES   = None

print(f"BATCH_SIZE={BATCH_SIZE}, LR={LR}, EPOCHS={EPOCHS}, IMG_SIZE={IMG_SIZE}")



## 4. Dados — Oxford-IIIT Pet (*species*: **gato** ou **cachorro**)
Usaremos o `torchvision.datasets.OxfordIIITPet` e mapearemos o *target* **species** para binário: `0 = cat`, `1 = dog`.

**Por que esse dataset?** Porque é público, conhecido e pode ser baixado com `download=True` diretamente do PyTorch, evitando etapas externas.


In [ ]:

from functools import partial

# Transforms para treino/val
train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# target_types='species' retorna espécie (cat/dog). Vamos garantir 0/1.
def species_to_binary(target):
    # Em versões atuais do torchvision, species já é 0 (cat) ou 1 (dog).
    # Ainda assim, protegemos caso venha em outro formato.
    if isinstance(target, (list, tuple)):
        target = target[0]
    if isinstance(target, torch.Tensor):
        target = target.item()
    # Normaliza para {0,1}
    if target in (0, 1):
        return int(target)
    # Fallback: mapeia 1->0 (cat), 2->1 (dog) se vier em {1,2}
    if target in (1, 2):
        return int(target - 1)
    raise ValueError(f"Target species inesperado: {target}")

root = "/mnt/data/oxford-iiit-pet"
os.makedirs(root, exist_ok=True)

trainval_ds = OxfordIIITPet(
    root=root,
    split="trainval",
    target_types="binary-category",
    transform=train_tfms,
    target_transform=species_to_binary,
    download=True,
)

test_ds = OxfordIIITPet(
    root=root,
    split="test",
    target_types="binary-category",
    transform=val_tfms,
    target_transform=species_to_binary,
    download=True,
)

print("Tamanho (trainval):", len(trainval_ds), "| Tamanho (test):", len(test_ds))

# Sanity check: rótulos binários
ys = [trainval_ds[i][1] for i in range(min(1000, len(trainval_ds)))]
u = sorted(set(ys))
print("Valores únicos (amostra):", u)
assert_true(set(u).issubset({0,1}), "Targets binários (0=cat,1=dog) confirmados.", "Targets não são binários.")



## 5. *Split* Treino/Validação e *DataLoaders*
Complete as linhas marcadas para criar `train_ds`, `val_ds` e os respectivos `DataLoader`s.


In [ ]:

N = len(trainval_ds)
n_val = int(N * VAL_SPLIT)
n_train = N - n_val

### Inicie seu código aqui ###
# 1) Faça o split aleatório (com seed fixo para reprodutibilidade)

# 2) (Opcional) Subamostre para rodar mais rápido

# 3) Crie os DataLoaders de treino e validação

### Termine seu código aqui ###

# Sanity checks
xb, yb = next(iter(train_loader))
print("Batch shape:", xb.shape, yb.shape, "| dtype:", xb.dtype, yb.dtype)
assert_true(xb.shape[1] == 3 and xb.shape[-1] == IMG_SIZE, "Shapes ok para imagens RGB redimensionadas.", "Shapes inesperados.")
assert_true(set(yb.unique().tolist()).issubset({0,1}), "Rótulos no batch são binários.", "Rótulos inválidos no batch.")



## 6. Modelo: **ResNet18** com *fine-tuning* leve
Usaremos uma `resnet18` pré-treinada no ImageNet e trocaremos a camada final para **2** classes.


In [ ]:

from torchvision.models import resnet18, ResNet18_Weights

### Inicie seu código aqui ###

### Termine seu código aqui ###

# Sanity check: passada direta num batch
model.eval()
with torch.no_grad():
    out = model(xb.to(device))
print("Saída do modelo:", out.shape)
assert_true(out.shape[-1] == 2, "Camada final com 2 neurônios confirmada.", "Camada final não tem 2 saídas.")



## 7. Otimizador, Loss e Métricas


In [ ]:

### Inicie seu código aqui ###

### Termine seu código aqui ###

def accuracy_from_logits(logits, targets):
    preds = logits.argmax(dim=1)
    return (preds == targets).float().mean().item()



## 8. Loop de Treino/Validação
Preencha os trechos pedidos para completar uma época de treino e validação.


In [ ]:

def run_epoch(model, loader, optimizer=None):
    is_train = optimizer is not None
    model.train(is_train)
    total_loss, total_acc, total_count = 0.0, 0.0, 0

    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        if is_train:
            optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        if is_train:
            ### Inicie seu código aqui ###

            ### Termine seu código aqui ###
        with torch.no_grad():
            acc = accuracy_from_logits(logits, yb)
        bs = xb.size(0)
        total_loss += loss.item() * bs
        total_acc  += acc * bs
        total_count += bs
    return total_loss / total_count, total_acc / total_count

history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

best_val_acc = 0.0
start = time.time()
for epoch in range(1, 10):
    tr_loss, tr_acc = run_epoch(model, train_loader, optimizer)
    vl_loss, vl_acc = run_epoch(model, val_loader, optimizer=None)

    history["train_loss"].append(tr_loss)
    history["train_acc"].append(tr_acc)
    history["val_loss"].append(vl_loss)
    history["val_acc"].append(vl_acc)

    print(f"[{epoch:02d}/{EPOCHS}] train_loss={tr_loss:.4f} | train_acc={tr_acc*100:.1f}%  || val_loss={vl_loss:.4f} | val_acc={vl_acc*100:.1f}%")

end = time.time()
print(f"Tempo total: {end-start:.1f}s")

# Sanity check: perda não explode e acurácia > 50% (mínimo razoável)
assert_true(np.isfinite(history['train_loss'][-1]), "Treino concluído com perdas finitas.", "Perda não finita.")
assert_true(history['val_acc'][-1] >= 0.5, "Acurácia de validação ≥ 50% atingida.", "Acurácia abaixo de 50%. Revise hiperparâmetros/execução.")



## 9. Curvas de Treino/Validação


In [ ]:

import matplotlib.pyplot as plt

# Perda
plt.figure()
plt.plot(history["train_loss"], label="train_loss")
plt.plot(history["val_loss"], label="val_loss")
plt.xlabel("Época"); plt.ylabel("Perda"); plt.title("Curva de Perda"); plt.legend(); plt.show()

# Acurácia
plt.figure()
plt.plot(history["train_acc"], label="train_acc")
plt.plot(history["val_acc"], label="val_acc")
plt.xlabel("Época"); plt.ylabel("Acurácia"); plt.title("Curva de Acurácia"); plt.legend(); plt.show()



## 10. Avaliação no *Test Set*


In [ ]:

test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

test_loss, test_acc = run_epoch(model, test_loader, optimizer=None)
print(f"Teste — loss={test_loss:.4f} | acc={test_acc*100:.1f}%")

# Sanity check final
assert_true(test_acc >= 0.5, "Acurácia de teste ≥ 50% (baseline) atingida.", "Acurácia de teste abaixo do baseline.")



## 11. Amostras de Predição


In [ ]:

idxs = np.random.choice(len(test_ds), size=min(8, len(test_ds)), replace=False)
model.eval()
inv_norm = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225]
)

class_names = ["cat", "dog"]

plt.figure(figsize=(12, 6))
for i, idx in enumerate(idxs, 1):
    img, y = test_ds[idx]
    with torch.no_grad():
        logits = model(img.unsqueeze(0).to(device))
        pred = logits.argmax(dim=1).item()

    img_viz = inv_norm(img).clamp(0,1).permute(1,2,0).cpu().numpy()
    plt.subplot(2, 4, i)
    plt.imshow(img_viz)
    plt.axis("off")
    title = f"GT: {class_names[y]} | Pred: {class_names[pred]}"
    plt.title(title)
plt.tight_layout(); plt.show()



## 12. O que entregar
- **Notebook executado** com todas as células, *sanity checks* passando e métricas reportadas.
- Pontos de edição foram limitados às seções **claramente marcadas** para evitar ambiguidade.
- (Opcional) Compare resultados ao alterar `BATCH_SIZE`, `LR`, `EPOCHS` e registre observações.
